# Output Master

In [2]:
def OutputOptionsMaster(master_df, avgVol_opt, rsi_opt, fip_opt, cr_opt, beta_opt, roa_opt, dcf_opt, historical_datas_fromInterface, quality_df=None):
  import pandas as pd
  temp_df = master_df.copy()

  if cr_opt == True:
    temp_df = CumulativeReturn(temp_df, historical_datas_fromInterface)
  
  if fip_opt == True:
    temp_df = FIPOutputOption(temp_df, historical_datas_fromInterface)


  # Merge quality_df with temp_df
  if quality_df != None:
    final_merged = pd.merge(temp_df, quality_df, on=['Symbol'])
    final_merged.dropna()
    print(final_merged)
  else: 
    temp_df = QualityOutput(temp_df)
    final_merged = temp_df.copy()
    final_merged.dropna()

  # Final Check To Get Desired Columns
  desiredColumns = ['Symbol', 'Name', 'QUALITY', 'Industry','MarketCapSize']
  if avgVol_opt == True:
    desiredColumns.append('Volume(avg)')
  if fip_opt == True:
    desiredColumns.append('FIP')
  if cr_opt == True:
    desiredColumns.append('CR(14d)')
  if beta_opt == True:
    desiredColumns.append('beta')
  if roa_opt == True:
    desiredColumns.append('ROA')
  if dcf_opt == True:
    desiredColumns.append("dcf")
  
  print(desiredColumns)
  final_merged = final_merged[desiredColumns]
  print(final_merged)
  return final_merged


# Cumulative Return Option

In [ ]:
def CumulativeReturn(master_df, historical_datas):
  temp_df = master_df.copy()


  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()
  offset = BMonthEnd(-1)
  offsetFirst = BMonthBegin(-13)

  # This will be last day of the previous month
  #lastDayLastMonth = todayD+offset

  # This will be one year ago for current month
  firstDayFirstMonth = todayD+offsetFirst

  from dateutil.relativedelta import relativedelta, FR, MO, TH
  from datetime import datetime
  from datetime import date
  import pandas as pd
  import numbers

  failureCount = 0

  d=date.today()
  #prev 1 dates
  prev1Fri = d + relativedelta(weekday=FR(-1))
  prev1Fri = prev1Fri.strftime("%Y-%m-%d")
  prev1Fri = prev1Fri + " 00:00:00"
  prev1Fri = pd.to_datetime(prev1Fri)

  prev1Mon = d + relativedelta(weekday=MO(-2))
  prev1Mon = prev1Mon.strftime("%Y-%m-%d")
  prev1Mon = prev1Mon + " 00:00:00"
  prev1Mon = pd.to_datetime(prev1Mon)

  prev2Fri = d + relativedelta(weekday=FR(-2))
  prev2Fri = prev2Fri.strftime("%Y-%m-%d")
  prev2Fri = prev2Fri + " 00:00:00"
  prev2Fri = pd.to_datetime(prev2Fri)

  prev2Mon = d + relativedelta(weekday=MO(-3))
  prev2Mon = prev2Mon.strftime("%Y-%m-%d")
  prev2Mon = prev2Mon + " 00:00:00"
  prev2Mon = pd.to_datetime(prev2Mon)

  firstDates = [prev1Mon, prev2Mon]
  lastDates = [prev1Fri, prev2Fri]


  #
  #  Put Historical Prices into dataframe
  #
  #
  # Append prices to tickers for given dataframe screen
  prices_df = pd.DataFrame()


  # Historical_datas holds the year worth of data. 
  # Here we will be iterating over the tickers that passed the volume screen to get their prices from one year ago, till today.
  for ticker in temp_df['Symbol'].to_list():
    try:
      prices_df[ticker] = historical_datas[ticker]['close'].loc[f"{firstDayFirstMonth.year}-{firstDayFirstMonth.month}-{firstDayFirstMonth.day}" : f"{todayD.year}-{todayD.month}-{todayD.day}"]
    except:
      failureCount +=1

  # Drop any that did not work
  prices_df.dropna()


  import time 
  import datetime 

  pricesLessThan1TickerList = list(prices_df.columns)

  CR_dict = {}
  for CRticker in pricesLessThan1TickerList:
    tempList = []
    for x in range(0,len(firstDates)):
      
      # Get First Date Value
      try:
        doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day}"]
      except:
        try:
          doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day+1}"]
        except:
          try:
            doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day+2}"]
          except:
            try:
              doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day+3}"]
            except:
              print("Failure in First Date Value")
              failureCount +=1
      # Get Last Date Value
      try:
        doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day}"]
      except:
        try:
          doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day-1}"]
        except:
          try:
            doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day-2}"]
          except:
            try:
              doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day-3}"]
            except:
              print("Failure in Last Date Value")
              failureCount +=1
        ###
      CRValue = ((doubleLast - doubleFirst)/doubleFirst)+1
      tempList.append(CRValue)
    #Append to dict
    CR_dict[CRticker] = tempList


  import numpy
  final_CR_dict = {}

  for finalTickers in pricesLessThan1TickerList:
    tempList = CR_dict[finalTickers]
    final_CR_dict[finalTickers] = numpy.prod(tempList)-1

  print(final_CR_dict)



  # Create final CR data frame
  CR_df = pd.DataFrame(list(final_CR_dict.items()),columns = ['Symbol','CR(14d)'])

  # Merge final CR data frame with master df passed in parameter
  master_df_copy = master_df.copy()
  inner_merge_df = pd.merge(master_df_copy, CR_df, on=['Symbol'])
  inner_merge_df.dropna()
  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()
  print(f"Failure Count 14d CR = {failureCount}")
  print(inner_merge_df)
  return inner_merge_df


# FIP Output Option

In [ ]:
def FIPOutputOption(initial_df, historical_datas):
  temp_df = initial_df.copy()
  from yahoofinancials import YahooFinancials
  from yahoo_fin.stock_info import get_data
  from yahoo_finance import Share
  import pandas as pd

  # Get previous months dates
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin
  from datetime import date


  # Date Range:
  firstDates = []
  lastDates = []
  d=date.today()

  for x in range(1,13):
    offset = BMonthEnd(-x)
    offsetFirst = BMonthBegin(-x-1)
    firstDates.append(d+offsetFirst)
    lastDates.append(d+offset)

############################################################################################################################################################################################################################################################################################
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()
  offset = BMonthEnd(-1)
  offsetFirst = BMonthBegin(-13)

  # This will be last day of the previous month
  #lastDayLastMonth = todayD+offset

  # This will be one year ago for current month
  firstDayFirstMonth = todayD+offsetFirst

  from dateutil.relativedelta import relativedelta, FR, MO, TH
  from datetime import datetime
  from datetime import date
  import pandas as pd
  import numbers

  failureCount = 0

  d=date.today()
  #prev 1 dates
  prev1Fri = d + relativedelta(weekday=FR(-1))
  prev1Fri = prev1Fri.strftime("%Y-%m-%d")
  prev1Fri = prev1Fri + " 00:00:00"
  prev1Fri = pd.to_datetime(prev1Fri)

  prev1Mon = d + relativedelta(weekday=MO(-2))
  prev1Mon = prev1Mon.strftime("%Y-%m-%d")
  prev1Mon = prev1Mon + " 00:00:00"
  prev1Mon = pd.to_datetime(prev1Mon)

  prev2Fri = d + relativedelta(weekday=FR(-2))
  prev2Fri = prev2Fri.strftime("%Y-%m-%d")
  prev2Fri = prev2Fri + " 00:00:00"
  prev2Fri = pd.to_datetime(prev2Fri)

  prev2Mon = d + relativedelta(weekday=MO(-3))
  prev2Mon = prev2Mon.strftime("%Y-%m-%d")
  prev2Mon = prev2Mon + " 00:00:00"
  prev2Mon = pd.to_datetime(prev2Mon)

  firstDates = [prev1Mon, prev2Mon]
  lastDates = [prev1Fri, prev2Fri]


  #
  #  Put Historical Prices into dataframe
  #
  #
  # Append prices to tickers for given dataframe screen
  prices_df = pd.DataFrame()


  # Historical_datas holds the year worth of data. 
  # Here we will be iterating over the tickers that passed the volume screen to get their prices from one year ago, till today.
  for ticker in temp_df['Symbol'].to_list():
    try:
      prices_df[ticker] = historical_datas[ticker]['close'].loc[f"{firstDayFirstMonth.year}-{firstDayFirstMonth.month}-{firstDayFirstMonth.day}" : f"{todayD.year}-{todayD.month}-{todayD.day}"]
    except:
      failureCount +=1

  # Drop any that did not work
  prices_df.dropna()



#############################################################################################################################################################################################################################################################################
  tickerList = temp_df['Symbol'].to_list()

  CR_dict = {}
  for CRticker in tickerList:
    tempList = []
    for x in range(0,len(firstDates)):
      # Get First Date Value
      try:
        doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day}"]
      except:
        try:
          doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day+1}"]
        except:
          try:
            doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day+2}"]
          except:
            try:
              doubleFirst = prices_df[CRticker].loc[f"{firstDates[x].year}-{firstDates[x].month}-{firstDates[x].day+3}"]
            except:
              print("Failure in First Date Value")
              failureCount +=1
      # Get Last Date Value
      try:
        doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day}"]
      except:
        try:
          doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day-1}"]
        except:
          try:
            doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day-2}"]
          except:
            try:
              doubleLast = prices_df[CRticker].loc[f"{lastDates[x].year}-{lastDates[x].month}-{lastDates[x].day-3}"]
            except:
              print("Failure in Last Date Value")
              failureCount +=1
        ###
      CRValue = ((doubleLast - doubleFirst)/doubleFirst)+1
      tempList.append(CRValue)
    #Append to dict
    CR_dict[CRticker] = tempList
  # Create new data frame to hold CR's. Create list of tickers from prices_df

  print(CR_dict)



  import numpy

  final_CR_dict = {}

  for finalTickers in tickerList:
    tempList = CR_dict[finalTickers]
    final_CR_dict[finalTickers] = numpy.prod(tempList)-1


  print(final_CR_dict)



  # Get number of times up and down for the year
  # Use prices_df and get count of num pos days and num neg days
  # Create a dictionary to hold the pos and neg number

  prices_df.head(15)
  posNeg_dict = {}

  print(len(prices_df))

# Get counts for pos/neg times in the past year
  for posnegTickers in prices_df.columns:
    posNeg_list = []
    # Set entire column values to list
    posNeg_list = prices_df[posnegTickers].tolist()
    results_list = []
    for x in range(0,len(posNeg_list)):
      #calculate (next - current)/current
      try:
        temp = ((posNeg_list[x+1] - posNeg_list[x])/posNeg_list[x])
        results_list.append(temp)
      except:
        #do nothing
        fake = -1
    #Count num pos and num neg
    pos_count, neg_count = 0,0
    for num in results_list:
      if num >= 0:
        pos_count +=1
      else:
        neg_count +=1
    
    #use dict to hold {ticker: [pos,neg]}
    posNeg_dict[posnegTickers] = [pos_count, neg_count]


# FIP Calculation
  # iterate through each ticker and complete the final calculation
  fip_dict = {}

  temptempList = posNeg_dict['A']
  print(temptempList)
  for fipTickers in prices_df.columns:
    fipTempList = posNeg_dict[fipTickers]
    negPercent = fipTempList[1]/(fipTempList[0] + fipTempList[1])
    posPercent = fipTempList[0]/(fipTempList[0] + fipTempList[1])
    if final_CR_dict[fipTickers] > 0:
      signPRET = 1
    else:
      signPRET = -1
    fip_dict[fipTickers] = (signPRET * (negPercent - posPercent))

  print(fip_dict)



# Dict to Data Frames
  CR_df = pd.DataFrame(list(final_CR_dict.items()),columns = ['Symbol','CR(12M)'])
  FIP_df = pd.DataFrame(list(fip_dict.items()),columns = ['Symbol','FIP'])


  # print(FIP_df)
  # print(CR_df)



  # Merge final CR data frame with master df passed in parameter
  master_df_copy = initial_df.copy()
  inner_merge_df = pd.merge(master_df_copy, CR_df, on=['Symbol'])
  inner_merge_df.dropna()

  # Merge FIP df with inner Merge from CR
  fip_df_merged = pd.merge(master_df_copy, FIP_df, on=['Symbol'])
  fip_df_merged.dropna()

  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()
  print(f"Failure Count FIP: {failureCount}")
  print(fip_df_merged)
  return fip_df_merged

# Quality Output

In [1]:
def QualityOutput(initial_df):
  from yahoofinancials import YahooFinancials
  from yahoo_fin.stock_info import get_data
  from yahoo_finance import Share
  import pandas as pd
  import numpy as np
  import math
  import time
  ticker_list = initial_df['Symbol'].to_list()
  failureCount = 0
# Start by creating df to hold all required variables
#
#
  # Create dataframe headers
  # Stock's ticker should be index

  # stock_list = ['MSFT', 'AAPL', 'AMZN', 'GOOG', 'WMT', 'PG', 'SKT', 'BBBY', 'HD', 'DIS', 'LOGI', 'GE','F', 'CAT', 'INTC']

  quantitative_df = pd.DataFrame(index=ticker_list, columns= ['PRICE','ROA','PERCENTILE_ROA','ROC','PERCENTILE_ROC','FCFA','PERCENTILE_FCFA','MG','PERCENTILE_MG','MS','PERCENTILE_MS','MM','FP','PERCENTILE_FP','FS_ROA','FCFTA','FS_FCFTA','ACCRUAL','FS_ACCRUAL','LEVER','FS_LEVER','LIQUID','FS_LIQUID','NEQISS','FS_NEQISS','DELTA_ROA','FS_DELTA_ROA','DELTA_FCFTA','FS_DELTA_FCFTA','DELTA_MARGIN','FS_DELTA_MARGIN','DELTA_TURN','FS_DELTA_TURN','P_FS','QUALITY'])
  print(quantitative_df)
########################################################################################################################################################################################################################################################################################################################
# Begin Quantitative Value Screening
#
#
  finanacials_api_key = "d8955c763cf41e8593a8bd57288aa7d3"
#
  !pip install requests
  import requests

  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()
  
  # Finding the cheapest stocks
  # Price = EBIT / TEV

  # (Stock_list) is for testing purposes (ticker_list is for final output)
  limit = 8


  count = 0
  for stock in ticker_list:
    count+=2

    if count >= 290:
      print(count)
      print("Halting for api key timer to refresh")
      count = 1
      time.sleep(61)

    # Request is 'Company Key Metrics' for enterpriseValueOverEBITDATTM, which is ebit
    CFS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{stock}?limit={limit}&apikey={finanacials_api_key}');
    cfs_dict= CFS.json()

    # Request is 'Compnay Enterprise Value' for enterpriseValue, which is TEV
    CEV = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{stock}?limit={limit}&apikey={finanacials_api_key}');
    cev_dict = CEV.json()


    # Access elements through json
    try: 
      ebit = cfs_dict[0]['ebitda']
      # print(f'Ebit for {stock} is: {ebit}')
    except:
      # print(f"Issue with {stock}")
      failureCount += 1

    try:
      tev = cev_dict[0]['enterpriseValue']
      # print(f'TEV for {stock} is: {tev}')
    except:
      # print(f"Issue with {stock}")
      failureCount += 1

    try:
      price = ebit/tev
      # print(f"Price for {stock} is: {price}")
    except:
      # print(f"Issue with {stock}")
      failureCount += 1

    # Insert Price into DataFrame
    #print(f"{stock}'s ebitda = {ebit}, {stock}'s ev = {tev}")
    quantitative_df['PRICE'].loc[f"{stock}"] = price

  print(quantitative_df)

########################################################################################################################################################################################################################################################################################################################

#   Finding Highest Quality Stocks
  import numpy as np
  import math as mt

  requestYearlyLimit = 8

  for stock in ticker_list:
    count+=4

    if count >= 290:
      print(count)
      count = 1
      time.sleep(61)

    # Company Financial Ratios
    CFR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{stock}?limit={requestYearlyLimit}&apikey={finanacials_api_key}');
    # CFR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios-ttm/{stock}?apikey={finanacials_api_key}');
    cfr_dict = CFR.json()

    # Cash Flow Statment
    cashFS = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{stock}?limit={requestYearlyLimit}&apikey={finanacials_api_key}')
    cashFS_dict = cashFS.json()

    # Below request is for 'Company Financial Statements'
    CFS =requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{stock}?limit={requestYearlyLimit}&apikey={finanacials_api_key}');
    cfs_dict = CFS.json()

    # Balanace Spread Sheet
    BSS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{stock}?limit={requestYearlyLimit}&apikey={finanacials_api_key}');
    bss_dict = BSS.json()  

    #print("Lists Reset")
    eightYearROA_list = []
    eightYearROC_list = []
    eightYearFCFA_list = []
    eightYearMG_list = []
    eightYearMS_list = []

    companyAge = -1

    for x in range(0,8):
      try: 
        # ROA
        t_ROA = cfr_dict[x]['returnOnAssets'] + 1
        #print(t_ROA)
        eightYearROA_list.append(t_ROA)
        companyAge = x
      except: 
        if companyAge == -1:
          print(f"{stock} is less than 1 year old")
          failureCount += 1
      try:
        # ROC
        t_ROC = cfr_dict[x]['returnOnCapitalEmployed'] + 1
        eightYearROC_list.append(t_ROC)
        companyAge = x
      except:
        if companyAge == -1:
          print(f"{stock} is less than 1 year old")
      try:
        # FCFA
        t_FCFA = cashFS_dict[x]['freeCashFlow']
        eightYearFCFA_list.append(t_FCFA + 1)
        companyAge = x
      except:
        if companyAge == -1:
          print(f"{stock} is less than 1 year old")
      try:
        # MG
        t_MG = cfr_dict[x]['grossProfitMargin'] + 1
        eightYearMG_list.append(t_MG + 1)
        companyAge = x
      except:
        if companyAge == -1:
          print(f"{stock} is less than 1 year old")
      try:
        # MS NEEDS TO BE UPDATED AND COMPLETED
        t_MS = (cfs_dict[x]['revenue'] - cfs_dict[x]['costOfRevenue']) / (cfs_dict[x]['revenue'])
        eightYearMS_list.append(t_MS + 1)
        companyAge = x
      except:
        if companyAge == -1:
          print(f"{stock} is less than 1 year old")


    try:
      n_roa = 1/len(eightYearROA_list)
    except:
      print('non div zero')

    try:
      n_roc = 1/len(eightYearROC_list)
    except:
      print('non div zero')


    try:
      n_MG = 1/len(eightYearMG_list)
    except:
      print('non div zero')


    try:

      eightYearROA = ( ( ( np.power(np.prod(eightYearROA_list), n_roa) ) -1) * 100 )
      #print(f"8 year roa is: {eightYearROA}")
      eightYearROC = ( ( ( np.power(np.prod(eightYearROC_list), n_roc) ) -1)* 100 ) 
      eightYearFCFA = np.sum(eightYearFCFA_list) / (bss_dict[0]['totalAssets'] * len(eightYearFCFA_list))
      eightYearMG = ( ( ( np.power(np.prod(eightYearMG_list), n_MG)) -1)* 100 )
      eightYearMS_avg = (np.average(eightYearMS_list))
    except:
      print('non div somewhere')
      
    # Put Data In Table
    quantitative_df['ROA'].loc[f"{stock}"] = eightYearROA
    quantitative_df['ROC'].loc[f"{stock}"] = eightYearROC
    quantitative_df['FCFA'].loc[f"{stock}"] = eightYearFCFA
    quantitative_df['MG'].loc[f"{stock}"] = eightYearMG
    quantitative_df['MS'].loc[f"{stock}"] = eightYearMS_avg


    #print(f"{stock}'s {len(eightYearROA_list)} Year ROA = {eightYearROA} %")
    #print(f"{stock}'s {len(eightYearROC_list)} Year ROC = {eightYearROC} %")
    #print(f"{stock}'s {len(eightYearMG_list)} Year MG = {eightYearMG} %")
    #print(f"{stock}'s {len(eightYearFCFA_list)} Year FCFA = {eightYearFCFA}")
    #print(f"{stock}'s {len(eightYearFCF_list)} Year MS Avg = {eightYearMS_avg * 100} %")
    #print(f"{stock}'s Margin Max = {MM}")
    #print(f"{stock}'s Percentile Franchise Power = {p_FP}")
    #print("")
########################################################################################################################################################################################################################################################################################################################
########################################################################################################################################################################################################################################################################################################################
  
  
# Ranking Percentiles

  quantitative_df['PERCENTILE_ROA'] = quantitative_df.ROA.rank(pct = True)
  quantitative_df['PERCENTILE_ROC'] = quantitative_df.ROC.rank(pct = True)
  quantitative_df['PERCENTILE_FCFA'] = quantitative_df.FCFA.rank(pct = True)
  quantitative_df['PERCENTILE_MG'] = quantitative_df.MG.rank(pct = True)
  quantitative_df['PERCENTILE_MS'] = quantitative_df.MS.rank(pct = True)

  ## Margin Max (MM) Max(percentile_MG, percentile_MS)
  for stock in ticker_list:
    # Take max number of stock's
    try:
      quantitative_df['MM'].loc[f"{stock}"] = max(quantitative_df['PERCENTILE_MG'].loc[f"{stock}"] , quantitative_df['PERCENTILE_MS'].loc[f"{stock}"])
    except:
      print(f'not enough data for stock {stock}')
      failureCount += 1

  # Franchise Power
  for stock in ticker_list:
    temp_list = []
    sum = 0

    try:
      temp_list.append(quantitative_df['PERCENTILE_ROA'].loc[f"{stock}"])
      temp_list.append(quantitative_df['PERCENTILE_ROC'].loc[f"{stock}"])
      temp_list.append(quantitative_df['PERCENTILE_FCFA'].loc[f"{stock}"])
      temp_list.append(quantitative_df['MM'].loc[f"{stock}"])

        # sum up elements
      for ele in temp_list:
        sum += ele
      
      t_p_fp = sum / len(temp_list) 
      #print(f"average of {stock}'s numbers is: {t_p_fp}")
      quantitative_df['FP'].loc[f"{stock}"] = t_p_fp
      
    except:
      print(f'not enough data for stock {stock}')
      failureCount += 1



  # Percentile Ranking Franchise Power
  quantitative_df['PERCENTILE_FP'] = quantitative_df.FP.rank(pct = True)


########################################################################################################################################################################################################################################################################################################################
# Net income before extraordinary items (t) / total assets(t)

  recentFilingLimit = 2


  for stock in ticker_list:
    count+=3

    if count >= 290:
      print(count)
      count = 1
      time.sleep(61)


    # Company Financial Ratios (Annual)
    CFR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{stock}?limit={recentFilingLimit}&apikey={finanacials_api_key}');
    # CFR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios-ttm/{stock}?apikey={finanacials_api_key}');
    cfr_dict = CFR.json()

    # Cash Flow Statment (Annual)
    cashFS = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{stock}?limit={recentFilingLimit}&apikey={finanacials_api_key}')
    cashFS_dict = cashFS.json()

    # Balance Sheet Statement
    # Limits refer to the number of statements asking for starting from most recent backwards
    BSS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{stock}?limit={recentFilingLimit}&apikey={finanacials_api_key}');
    bss_dict = BSS.json()

    try:
      # Roa
      roa = cfr_dict[0]['returnOnAssets']
      if roa > 0:
        fs_ROA = 1
      else:
        fs_ROA = 0
      
      # Fcfta
      fcfta = cashFS_dict[0]['freeCashFlow'] / bss_dict[0]['totalAssets']
      if fcfta > 0:
        fs_FCFTA = 1
      else:
        fs_FCFTA = 0
      
      # Accrual
      accrual = fcfta - roa 
      if accrual > 0:
        fs_ACCRUAL = 1
      else:
        fs_ACCRUAL = 0

      # Lever 
      lever = ( bss_dict[1]['longTermDebt'] / bss_dict[1]['totalAssets'] ) - ( bss_dict[0]['longTermDebt'] / bss_dict[0]['totalAssets'])
      if lever > 0:
        fs_LEVER = 1
      else:
        fs_LEVER = 0

      # Liquid
      liquid = cfr_dict[0]['currentRatio'] - cfr_dict[1]['currentRatio']
      if liquid > 0:
        fs_LIQUID = 1
      else:
        fs_LIQUID = 0

      # Neqiss (purchases are represented as negative. Using addition for calculation in this case)
      neqiss = cashFS_dict[0]['commonStockRepurchased'] + cashFS_dict[0]['commonStockIssued']
      #print(f"PURCHASED = {cashFS_dict[0]['commonStockRepurchased']}   ISSUED = {cashFS_dict[0]['commonStockIssued']}")
      if neqiss > 0:
        fs_NEQISS = 1
      else:
        fs_NEQISS = 0

      # Delta_Roa (current - previous)
      delta_ROA = cfr_dict[0]['returnOnAssets'] - cfr_dict[1]['returnOnAssets']
      if delta_ROA > 0:
        fs_DELTA_ROA = 1
      else:
        fs_DELTA_ROA = 0

      # Delta_Fcfta (current - previous)
      delta_FCFTA = ( cashFS_dict[0]['freeCashFlow'] / bss_dict[0]['totalAssets'] ) - cashFS_dict[1]['freeCashFlow'] / bss_dict[1]['totalAssets']
      if delta_FCFTA > 0:
        fs_DELTA_FCFTA = 1
      else:
        fs_DELTA_FCFTA = 0  

      # Delta_Margin
      delta_margin = cfr_dict[0]['grossProfitMargin'] - cfr_dict[1]['grossProfitMargin']
      if delta_margin > 0:
        fs_DELTA_MARGIN = 1
      else:
        fs_DELTA_MARGIN = 0  

      # Delta_Turn
      delta_TURN = cfr_dict[0]['assetTurnover'] - cfr_dict[1]['assetTurnover']
      if delta_TURN > 0:
        fs_DELTA_TURN = 1
      else:
        fs_DELTA_TURN = 0 

      # Financial Strength
      fs_list_total = [int(fs_ROA), int(fs_FCFTA), int(fs_ACCRUAL), int(fs_LEVER), int(fs_LIQUID), int(fs_NEQISS), int(fs_DELTA_ROA), int(fs_DELTA_FCFTA), int(fs_DELTA_MARGIN), int(fs_DELTA_TURN)]
      p_FS = np.sum(fs_list_total) / 10

      # Financial Power
      p_FP = quantitative_df['PERCENTILE_FP'][f'{stock}']


      # Identify Quality = 0.5 * Franchise power  + 0.5 Financial Strength
      quality = (0.5 * p_FS) + (0.5 * p_FP)

      #
      # Print and Inserts Below
      #

      #print(f"{stock}'s ROA = {roa}")
      #print(f"{stock}'s fs_ROA = {fs_ROA}")
          # Insert Data into DataFrame
      quantitative_df['FS_ROA'].loc[f"{stock}"] = fs_ROA

      #print(f"{stock}'s fcfta = {fcfta}")
      #print(f"{stock}'s fs_FCFTA = {fs_FCFTA}")
            # Insert Data into DataFrame
      quantitative_df['FCFTA'].loc[f"{stock}"] = fcfta
      quantitative_df['FS_FCFTA'].loc[f"{stock}"] = fs_FCFTA

      #print(f"{stock}'s accrual = {accrual}")
      #print(f"{stock}'s fs_ACCRUAL = {fs_ACCRUAL}")
            # Insert Data into DataFrame
      quantitative_df['ACCRUAL'].loc[f"{stock}"] = accrual
      quantitative_df['FS_ACCRUAL'].loc[f"{stock}"] = fs_ACCRUAL

      #print(f"{stock}'s lever = {lever}")
      #print(f"{stock}'s fs_LEVER = {fs_LEVER}")
            # Insert Data into DataFrame
      quantitative_df['LEVER'].loc[f"{stock}"] = lever
      quantitative_df['FS_LEVER'].loc[f"{stock}"] = fs_LEVER

      #print(f"{stock}'s liquid = {liquid}")
      #print(f"{stock}'s fs_LIQUID = {fs_LIQUID}")
            # Insert Data into DataFrame
      quantitative_df['LIQUID'].loc[f"{stock}"] = liquid
      quantitative_df['FS_LIQUID'].loc[f"{stock}"] = fs_LIQUID

      #print(f"{stock}'s neqiss = {neqiss}")
      #print(f"{stock}'s fs_NEQISS = {fs_NEQISS}")
            # Insert Data into DataFrame
      quantitative_df['NEQISS'].loc[f"{stock}"] = neqiss
      quantitative_df['FS_NEQISS'].loc[f"{stock}"] = fs_NEQISS

      #print(f"{stock}'s delta_ROA = {delta_ROA}")
      #print(f"{stock}'s fs_DELTA_ROA = {fs_DELTA_ROA}")
            # Insert Data into DataFrame
      quantitative_df['DELTA_ROA'].loc[f"{stock}"] = delta_ROA
      quantitative_df['FS_DELTA_ROA'].loc[f"{stock}"] = fs_DELTA_ROA

      #print(f"{stock}'s delta_FCFTA = {delta_FCFTA}")
      #print(f"{stock}'s fs_DELTA_FCFTA = {fs_DELTA_FCFTA}")
            # Insert Data into DataFrame
      quantitative_df['DELTA_FCFTA'].loc[f"{stock}"] = delta_FCFTA
      quantitative_df['FS_DELTA_FCFTA'].loc[f"{stock}"] = fs_DELTA_FCFTA

      #print(f"{stock}'s delta_margin = {delta_margin}")
      #print(f"{stock}'s fs_DELTA_MARGIN = {fs_DELTA_MARGIN}")
            # Insert Data into DataFrame
      quantitative_df['DELTA_MARGIN'].loc[f"{stock}"] = delta_margin
      quantitative_df['FS_DELTA_MARGIN'].loc[f"{stock}"] = fs_DELTA_MARGIN

      #print(f"{stock}'s delta_TURN = {delta_TURN}")
      #print(f"{stock}'s fs_DELTA_TURN = {fs_DELTA_TURN}")
            # Insert Data into DataFrame
      quantitative_df['DELTA_TURN'].loc[f"{stock}"] = delta_TURN
      quantitative_df['FS_DELTA_TURN'].loc[f"{stock}"] = fs_DELTA_TURN
      
      #print(f"{stock}'s Financial Strength (p_FS) = {p_FS}")
            # Insert Data into DataFrame
      quantitative_df['P_FS'].loc[f"{stock}"] = p_FS

      #print(f"{stock}'s Financial Power (p_FP) = {p_FP}")

      #print(f"{stock}'s Quality = {quality}")
            # Insert Data into DataFrame
      quantitative_df['QUALITY'].loc[f"{stock}"] = quality

    except:
      print(f'not enough data for stock {stock}')


    #print("")
########################################################################################################################################################################################################################################################################################################################
  quantitative_df = quantitative_df.sort_values(by=['QUALITY'], ascending= False)
  # Add new columns for additional Information
  quantitative_df_copy = quantitative_df.copy()
  quantitative_df_copy = quantitative_df_copy.dropna()


  quantitative_df_copy['industry'] = ""
  quantitative_df_copy['beta'] = ""
  quantitative_df_copy['range'] = ""
  quantitative_df_copy['dcf'] = ""
  quantitative_df_copy['mktCap'] = ""
  quantitative_df_copy['isETF'] = ""

  error_list = []
  for stocks in ticker_list:
    count+=1
    if count >= 290:
      print(count)
      count = 1
      time.sleep(61)
    
    CP = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{stocks}?apikey={finanacials_api_key}')
    cp_dict = CP.json() 

    try: 
      quantitative_df_copy['industry'].loc[f"{stocks}"] = cp_dict[0]['industry']
      quantitative_df_copy['beta'].loc[f"{stocks}"] = cp_dict[0]['beta']
      quantitative_df_copy['range'].loc[f"{stocks}"] = cp_dict[0]['range']
      quantitative_df_copy['dcf'].loc[f"{stocks}"] = cp_dict[0]['dcf']
      quantitative_df_copy['mktCap'].loc[f"{stocks}"] = cp_dict[0]['mktCap']
      quantitative_df_copy['isETF'].loc[f"{stocks}"] = cp_dict[0]['isEtf']
    except:
      print(f'error with {stocks}')
      failureCount += 1
      error_list.append(stocks)

  # Drop na's
  newColumns_df = quantitative_df_copy.dropna()
  print(newColumns_df)

  # Drop all the ETFS
  newColumns_df = newColumns_df[newColumns_df.isETF == newColumns_df.isETF.equals("False")]
  print(newColumns_df)


  # Add percentile
  master_df = newColumns_df.copy()
  master_df['Percentile_Quality'] = master_df.QUALITY.rank(pct = True)

  selected_columns = master_df[['PRICE','ROA','QUALITY','Percentile_Quality','industry','beta','range','dcf','mktCap']]
  quantitative_df_selected_columns = selected_columns.copy()
  quantitative_df_selected_columns.reset_index(inplace=True)
  quantitative_df_selected_columns = quantitative_df_selected_columns.rename(columns = {'index':'Symbol'})
  # print(quantitative_df_selected_columns)


  # Merge quantitative_df_select with initial df
  master_df_copy = initial_df.copy()
  quality_df_final = pd.merge(master_df_copy, quantitative_df_selected_columns, on=['Symbol'])
  quality_df_final.dropna()

  # Clear the outputs of the previous stuff
  clear_output()
  print(f"Failure Count in Quality: {failureCount}")
  print(quality_df_final)
  return quality_df_final